### Ingest qualifying json files

In [1]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

/home/ricardo/Documentos/Projects/dab-ci-cd/dab_project/.venv/lib/python3.12/site-packages/databricks/sdk/_widgets/__init__.py:71: UserWarning: 
To use databricks widgets interactively in your notebook, please install databricks sdk using:
	pip install 'databricks-sdk[notebook]'
Falling back to default_value_only implementation for databricks widgets.
  warnings.warn(


In [2]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lit

qualifying_schema = StructType(
    fields=[
        StructField("qualifyId", IntegerType(), False),
        StructField("raceId", IntegerType(), True),
        StructField("driverId", IntegerType(), True),
        StructField("constructorId", IntegerType(), True),
        StructField("number", IntegerType(), True),
        StructField("position", IntegerType(), True),
        StructField("q1", StringType(), True),
        StructField("q2", StringType(), True),
        StructField("q3", StringType(), True),
    ]
)

In [4]:
qualifying_df = (
    spark.read.schema(qualifying_schema)
    .option("multiLine", True)
    .json(f"{landing_folder_path}/{v_file_date}/qualifying")
)

##### Step 2 - Add ingestion meta data

In [5]:
qualifying_with_metadata_df = (
    add_ingestion_date(qualifying_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 - Write to output to bronze container

In [6]:
qualifying_with_metadata_df.write.mode("overwrite").format("delta").saveAsTable(
    "f1_bronze.qualifying"
)